In [35]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [36]:
ID = '1Cd-NSkZvJ0D1KMrRKZwawK875TCifQBjfmwqCYNAksI'
sheet1 = 'data'
sheet2 = 'enroll'
URL1 = f'https://docs.google.com/spreadsheets/d/{ID}/gviz/tq?tqx=out:csv&sheet={sheet1}'
URL2 = f'https://docs.google.com/spreadsheets/d/{ID}/gviz/tq?tqx=out:csv&sheet={sheet2}'

df = pd.read_csv(URL1)
enroll = pd.read_csv(URL2)

# --------------------
# master data frame
df = df.iloc[:,:7]
df['Stu_CloseContactAllowedOnCampus'] = df['Stu_CloseContactAllowedOnCampus'].astype('Int64')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date', ascending=True)


# ---------------------
# school enroll number, lat and long
enroll = enroll.iloc[:,:3]
enroll[['lat','long']] = enroll['lat_long'].str.split(',',expand=True)
enroll[['lat','long']] = enroll[['lat','long']].astype('float')
enroll['lat'] = np.round(enroll.lat, 4)
enroll['long'] = np.round(enroll.long, 4)

# ---------------------
# filter down to one specific school
df_schl = df[df['school']=='Mehlville High School'].sort_values('date')

# ---------------------
# filter down to current week
df_curr = df.copy().loc[df['date']==df['date'].max(), :]
df_curr = df_curr.merge(enroll[['school', 'lat', 'long', 'num_enroll']], on='school', how='left')
df_curr['total_new_case'] = df_curr.staff_newPos + df_curr.stu_newPos

# ---------------------
# group by week
df_weekSum = df.copy().groupby('date').sum().sort_values('date', ascending=True)

In [37]:
BARCHART_LAYOUT = dict(
    barmode='group',
    legend = dict(yanchor='top', y=0.99, xanchor='left',x=0.01),
    margin={"r":10,"t":10,"l":10,"b":10},
    width = 1000,
    height = 300,
    hovermode='x unified',
    hoverlabel=dict(bgcolor='white',font_size=10))

# staff new case and off-campus plot - Mehlville High School
plot_staf = go.Figure(data=[go.Bar(
    name = 'Staff New Case',
    x = df_schl.date,
    y = df_schl.staff_newPos.values
   ),
                       go.Bar(
    name = 'Staff off Campus',
    x = df_schl.date,
    y = df_schl.staff_offCampus
   )
])

plot_staf.update_layout(BARCHART_LAYOUT)
plot_staf.update_xaxes(showgrid=True, ticks='outside', tickson='boundaries', ticklen=5)
plot_staf.update_yaxes(tick0=0, dtick=1, title_text='Count')
                  
plot_staf.show()

In [38]:
# student plot - Melhville High School
plot_stud = go.Figure(data=[go.Bar(
    name = 'Student New Case',
    x = df_schl.date,
    y = df_schl.stu_newPos.values
   ),
                       go.Bar(
    name = 'Student off Campus',
    x = df_schl.date,
    y = df_schl.stu_offCampus
   )
])
plot_stud.update_layout(BARCHART_LAYOUT)
plot_stud.update_xaxes(showgrid=True, ticks='outside', tickson='boundaries', ticklen=5)
plot_stud.update_yaxes(tick0=0, dtick=5, title_text='Count')
                  
plot_stud.show()

In [39]:
# bar plot entire school district weekly new case, 
plot_week = go.Figure(data=[go.Bar(
    name = 'Weekly Total Student New Case',
    x = df_weekSum.index,
    y = df_weekSum.stu_newPos.values
   ),
                       go.Bar(
    name = 'Weekly Total Staff New Case',
    x = df_weekSum.index,
    y = df_weekSum.staff_newPos
   )
])
plot_week.update_layout(BARCHART_LAYOUT)
plot_week.update_xaxes(showgrid=True, ticks='outside', tickson='boundaries', ticklen=5)
plot_week.update_yaxes(tick0=0, dtick=10, title_text='Count')

plot_week.show()

In [40]:
# map 
plot_map = px.scatter_mapbox(
    df_curr, 
    lat="lat", lon="long", 
    hover_name="school",
    size = 'total_new_case',
    hover_data={'total_new_case':True, 'stu_newPos':True, 'staff_newPos':True, 'lat':False, 'long':False},
    color_discrete_sequence=["firebrick"], 
    zoom=11)
plot_map.update_layout(mapbox_style="open-street-map",
                       margin={"r":10,"t":10,"l":10,"b":10},
                       width=500, height=400)
plot_map.show()

In [43]:
df_weekSum

,staff_newPos,staff_offCampus,stu_newPos,stu_offCampus,Stu_CloseContactAllowedOnCampus
date,,,,,
2021-05-09,0,3,9,70,52
2021-05-16,0,1,7,74,18
2021-05-23,0,1,7,76,17
2021-05-30,0,0,3,13,11
2021-06-06,0,0,3,2,0
2021-06-13,0,0,0,0,0
2021-06-20,0,0,1,10,3
2021-07-04,0,0,3,6,1
2021-07-11,0,0,1,1,0


In [47]:
df_weekSum.iloc[-2,:].stu_newPos

58

In [54]:
#col1.metric('weekly student new case','9', '9')
studCase_new = df_weekSum.iloc[-1,:].stu_newPos
studCase_lastWk = df_weekSum.iloc[-2,:].stu_newPos
studCase_change = studCase_new-studCase_lastWk

#col2.metric('weekly student off-campus', '9', '9')
studOffc_new = df_weekSum.iloc[-1,:].stu_offCampus
studOffc_lastWk = df_weekSum.iloc[-2,:].stu_offCampus
studOffc_change = studOffc_new-studOffc_lastWk

#col3.metric('weekly staff new case', '9', '9')
stafCase_new = df_weekSum.iloc[-1,:].staff_newPos
stafCase_lastWk = df_weekSum.iloc[-2,:].staff_newPos
stafCase_change = stafCase_new-stafCase_lastWk

#col4.metric('weekly staff off-campus', '9', '9')
stafOffc_new = df_weekSum.iloc[-1,:].staff_offCampus
stafOffc_lastWk = df_weekSum.iloc[-2,:].staff_offCampus
stafOffc_change = stafOffc_new-stafOffc_lastWk

In [56]:
print(stafCase_new)
print(stafCase_lastWk)
print(stafCase_change)

6
4
2
